In [1]:
import faiss
import os
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

/home/lahirum/Documents/Mirada/RnD/FastApi/test2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")

/home/lahirum/Documents/Mirada/RnD/FastApi/test2/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
#load vector store
vector_store=FAISS.load_local("vector_store",embeddings,allow_dangerous_deserialization=True)

In [4]:
question = "What is the total face value of the convertible note?"

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from core.logger import logger


2025-10-22 12:55:14,780 - logger - INFO - Logger initialized.


In [6]:
embedding_model = "BAAI/bge-large-en"
vector_store_path= "vector_store"
retrieve_method ="max_marginal_relevance"


In [7]:
# ------------ Retrieval -----------------------------------------------------
def retrieve(retrieve_method:str):
    """
    Load FAISS vector store and return retriever.
    """
    logger.info(f"Initializing retriever using {embedding_model} and method: {retrieve_method}")
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
    logger.debug("Embeddings model loaded successfully.")

    #load vector store
    vector_store = FAISS.load_local(vector_store_path,embeddings,allow_dangerous_deserialization=True)
    retriever = vector_store.as_retriever(search_type="mmr")
    logger.debug(f"Vector store loaded from path: {vector_store_path}")


    return retriever


In [8]:

# --------------- Answer Generation -------------------------------------------
def generate_answer(question,retriever):
    """
    Generate an answer using RAG pipeline.
    """

    try:
        logger.info(f"Generating answer for question: {question}")

        retrieved_docs=retriever.invoke(question)
        print(f"retrieved_docs:{retrieved_docs}")
        context="\\n".join([d.page_content for d in retrieved_docs])

        logger.debug(f"retrived content: {context}")
        logger.info("documents retrieved successfully")
        
        # Define a chat prompt template composed of system and user messages
        chat_template= ChatPromptTemplate.from_messages([
        {"role": "system", "content": """Using the information contained in the context,
        give a comprehensive answer to the question.
        Respond only to the question asked, response should be concise and relevant to the question.
        Provide the number of the source document when relevant.
        If the answer cannot be deduced from the context, do not give an answer."""},
        {"role": "user", "content": f"""Context:
        {context}
        ---
        Now here is the question you need to answer.

        Question: {question}"""},
        ])

        logger.debug("Chat prompt template created.")

        logger.info("Initializing Ollama model...")
        llm= ChatOllama(
            model="llama3.2", 
            messages=chat_template,
            think=False,
            options={
                "num_predict": 4096,
                "temperature": 0.6,
                "top_p": 0.95,
                "top_k": 20,
                "repeat_penalty": 1.05,
            }    
        )
        logger.info("Ollama model initialized successfully.")

        chain = chat_template|llm
        answer = chain.invoke({"question": question})
        logger.info("Answer generated successfully.")
        print(answer.content)
        return answer.content , retrieved_docs
    except Exception as e:
        logger.exception(f"Error generating answer: {e}")
        raise
    

In [ ]:
from app.utils.rag_chain import generate_answer, retrieve, retrieve_method

In [10]:
question = "What is the total face value of the convertible note?"

In [11]:
retriever= retrieve(retrieve_method)

2025-10-22 12:55:22,127 - logger - INFO - Initializing retriever using BAAI/bge-large-en and method: max_marginal_relevance


In [40]:
answer, references = generate_answer(question,retriever)

2025-10-22 15:32:36,595 - logger - INFO - Generating answer for question: What is the total face value of the convertible note?
[Document(id='3e138805-138c-4410-b78c-2d1f75ee707e', metadata={'source': './uploaded_pdfs/sample.txt'}, page_content='| Net cash from/(used in) investing activities                                              | 2,578,486                | (489,175)                    |\n| Cash flows from financing activities                                                      |                          |                              |\n| Proceeds from the issue of shares                                                         | -                        | 618,460                      |\n| Payment of share issue costs                                                              | -                        | (22,500)                     |\n| Payment of convertible note issue costs                                                   | (71,446)                 | -                    

In [13]:
answer

'The total face value of the convertible note was $1,400,000.'

In [14]:
references

[Document(id='3e138805-138c-4410-b78c-2d1f75ee707e', metadata={'source': './uploaded_pdfs/sample.txt'}, page_content='| Net cash from/(used in) investing activities                                              | 2,578,486                | (489,175)                    |\n| Cash flows from financing activities                                                      |                          |                              |\n| Proceeds from the issue of shares                                                         | -                        | 618,460                      |\n| Payment of share issue costs                                                              | -                        | (22,500)                     |\n| Payment of convertible note issue costs                                                   | (71,446)                 | -                            |\n| Proceeds from borrowings - convertible notes payable                                      | 500,000                

In [15]:
from typing import List,Optional
from pydantic import BaseModel

In [16]:
class ChatRecord(BaseModel):
    """Response model for chat endpoint."""
    role: str
    content: str
    reference: Optional[List[str]] = []

In [17]:
class UserchatHistory(BaseModel):
    """Response model for user chat history."""
    user_id: int
    chat_history: list[ChatRecord]

In [18]:

chat_history = [
    {
        "user_id": 1,
        "chat_history": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Hello, how are you?"},
            {"role": "assistant", "content": "I'm doing well, thank you! How can I assist you today?","reference": []},
            {"role": "user", "content": "Tell me a joke."},
]
    },
    {
        "user_id": 2,
        "chat_history": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is the weather like today?"},
            {"role": "assistant", "content": "The weather is sunny with a high of 75°F.","reference": []},
            {"role": "user", "content": "Great, thank you!"},       
        ]
    },
    {
        "user_id": 3,
        "chat_history": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Can you help me with my homework?"},
            {"role": "assistant", "content": "Of course! What subject is your homework in?", "reference": []},
            {"role": "user", "content": "It's math."},
        ]
    },
    {
        "user_id": 4,
        "chat_history": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is the capital of France?"},
            {"role": "assistant", "content": "The capital of France is Paris.", "reference": []},
            {"role": "user", "content": "Thanks!"},
        ]
    },
    {
        "user_id": 5,
        "chat_history": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Can you recommend a good book?"},
            {"role": "assistant", "content": "Sure! 'To Kill a Mockingbird' is a great read.", "reference": []},
            {"role": "user", "content": "I've read that one. Something else?"},
        ]
    }
]  

In [ ]:
def add_chat_history(user_id: int, question: str, answer: str, reference: Optional[str] = None):
    """
    Add a new chat record to the chat history for a specific user.

    Args:
        user_id (int): The ID of the user.
        question (str): The user's question.
        answer (str): The assistant's answer.
        reference (Optional[str]): An optional reference for the answer.

    """
    try:
        clean_references = []
        # for ref in references:
            # if hasattr(ref, "page_content"):
            #     clean_references.append(ref.page_content)
            # else:
            #     clean_references.append(str(ref))
        for ref in references:
            clean_references.append(ref.page_content)
        for user in chat_history:
            if user["user_id"] == user_id:
                user["chat_history"].append({"role": "user", "content": question})
                user["chat_history"].append({"role": "assistant", "content": answer, "reference": clean_references})
                return 
    except Exception as e:
        logger.exception(f"Error : {e}")
        raise        


In [ ]:
# def add_chat_history(user_id, question, answer, references):
#     # Convert Document objects to strings 
#     clean_references = []
#     for ref in references:
#         if hasattr(ref, "page_content"):
#             clean_references.append(ref.page_content)
#         else:
#             clean_references.append(str(ref))

#     chat_entry = {
#         "question": question,
#         "answer": answer,
#         "reference": clean_references
#     }

In [30]:
user_input ={
    "user_id": 1,
    "question": question,

}

In [31]:
add_chat_history( user_input["user_id"],
        user_input["question"], answer,references )

In [37]:
user_id = 2

In [36]:
#Get chat history
from fastapi import FastAPI, HTTPException, status , Query
def get_chat_history(user_id: int) -> UserchatHistory:
    logger.info(f"chat history accessed for user_id={user_id}")
    """
    Get the chat history for a specific user.

    Args:
        user_id (int): The ID of the user whose chat history is to be retrieved.

    Returns:
        UserchatHistory: The chat history of the user.
    """
    for user in chat_history:
        if user["user_id"] == user_id:
            try:
                logger.debug(f"Returning chat history for user_id={user_id}")
                user_chat_history = UserchatHistory(user_id=user_id, chat_history=user["chat_history"])
                print(user_chat_history)
                return user_chat_history
            except Exception as e:
                logger.exception(f"Error retrieving chat history for user_id={user_id}: {e}")
                raise HTTPException(
                    status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
                    detail="An error occurred while retrieving chat history."
                )

    logger.warning(f"Chat history not found for user_id={user_id}")
    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail="User not found."
    )


In [41]:
from dataclasses import dataclass, field
from typing import List
@dataclass
class Student:
   name: str
   grades: List[int] = field(default_factory=list) # Use default_factory for mutable defaults
# Create new Student instances
